In [1]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.datasets import  OurRadiologsitsDataset, collate_fn
from our_radiologist.load import get_anns
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ap_ar
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.constants import full_iou_thrs, iou_thrs_5to95
from data.load  import seed_worker, get_dataloader_g
from tqdm import tqdm

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

c:\Users\mike8\Desktop\multimodal-abnormalities-detection\models\detectors\rcnn.py:102: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    CXR_Clinial_fusion1_fusion2="val_ar_0_5436_ap_0_1911_test_ar_0_5476_ap_0_3168_epoch49_WithClincal_05-23-2022 12-06-22_CXR_Clinical_roi_heads_spatialisation"
    CXR_Clinical_fusion1 = "val_ar_0_5476_ap_0_1984_test_ar_0_6038_ap_0_2757_epoch41_WithClincal_05-30-2022 08-01-54_CXR_Clinical_fusion1"
    CXR_Clinical_fusion2= "val_ar_0_4369_ap_0_2098_test_ar_0_4940_ap_0_2218_epoch58_WithClincal_05-30-2022 13-58-43_CXR_Clinical_fusion2"
    CXR="val_ar_0_5659_ap_0_1741_test_ar_0_5390_ap_0_1961_epoch36_WithoutClincal_05-29-2022 12-29-51_CXR"

In [4]:
normal_iou_thrs = iou_thrs_5to95
all_range_iou_thrs = full_iou_thrs
score_thresholds = [0.05]

In [5]:
import torch.nn as nn

In [6]:
import itertools

In [7]:

pre_spa_layers = itertools.chain.from_iterable([[
        nn.Linear(
        64,
        64,
        ),
        nn.BatchNorm1d(64),
        nn.LeakyReLU(),
    ]
    for _ in range(5)])

pre_spa = nn.Sequential(
    *pre_spa_layers
)

In [8]:
pre_spa.append(nn.Linear(64, 64),)

Sequential(
  (0): Linear(in_features=64, out_features=64, bias=True)
  (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=64, out_features=64, bias=True)
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=64, out_features=64, bias=True)
  (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): LeakyReLU(negative_slope=0.01)
  (9): Linear(in_features=64, out_features=64, bias=True)
  (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=64, out_features=64, bias=True)
  (13): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): LeakyReLU(negative_slope=0.01)
  (15): Linear(in_features=64, out_features=64, bias=True)
)

In [9]:
pre_spa

Sequential(
  (0): Linear(in_features=64, out_features=64, bias=True)
  (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=64, out_features=64, bias=True)
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=64, out_features=64, bias=True)
  (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): LeakyReLU(negative_slope=0.01)
  (9): Linear(in_features=64, out_features=64, bias=True)
  (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): LeakyReLU(negative_slope=0.01)
  (12): Linear(in_features=64, out_features=64, bias=True)
  (13): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): LeakyReLU(negative_slope=0.01)
  (15): Linear(in_features=64, out_features=64, bias=True)
)

# Run evaluation.

In [10]:
for select_model in tqdm([
    TrainedModels.CXR_Clinical_fusion1,
    TrainedModels.CXR_Clinical_fusion2,
    TrainedModels.CXR_Clinial_fusion1_fusion2,
    TrainedModels.CXR
]):

    for score_thrs in score_thresholds:

        model, train_info, _, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=score_thrs,
        )

        model.eval()

        iou_types = get_iou_types(model, train_info.model_setup)

        model_setup = train_info.model_setup
        dataset_params_dict = {
            "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
            "with_clinical": model_setup.use_clinical,
            "dataset_mode": model_setup.dataset_mode,
            "bbox_to_mask": model_setup.use_mask,
            "normalise_clinical_num": model_setup.normalise_clinical_num,
            "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
        }

        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=4,
        )

        train_coco, val_coco, test_coco = get_cocos(
            train_dataloader, val_dataloader, test_dataloader
        )

        radiologists_ann = get_anns("radiologists_annotated", detect_eval_dataset)

        radiologist_dataset = OurRadiologsitsDataset(
            detect_eval_dataset, radiologists_ann
        )
        radiologist_dataloader = torch.utils.data.DataLoader(
            radiologist_dataset,
            batch_size=4,
            shuffle=False,
            collate_fn=collate_fn,
            worker_init_fn=seed_worker,
            generator=get_dataloader_g(0),
        )

        radiologists_coco = get_coco_api_from_dataset(radiologist_dataloader.dataset)

        normal_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=normal_iou_thrs,
        )

        all_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=all_range_iou_thrs,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                all_range_eval_params_dict["bbox"].catIds = cat_ids
                all_range_eval_params_dict["segm"].catIds = cat_ids

            train_evaluator, _ = xami_evaluate(
                model,
                train_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=train_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            test_evaluator, _ = xami_evaluate(
                model,
                test_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            val_evaluator, _ = xami_evaluate(
                model,
                val_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=val_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            radiologist_evaluator, _ = xami_evaluate(
                model,
                radiologist_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=radiologists_coco,
                iou_types=iou_types,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            save_iou_results(
                train_evaluator,
                f"train_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                test_evaluator,
                f"test_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                val_evaluator,
                f"val_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                radiologist_evaluator,
                f"our_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )

            train_ap_ar = get_ap_ar(
                train_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            test_ap_ar = get_ap_ar(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            val_ap_ar = get_ap_ar(
                val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            our_ap_ar = get_ap_ar(
                radiologist_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            df = pd.DataFrame(
                [
                    {
                        "dataset": "train",
                        f"AP@[IoBB = 0.50]": train_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": train_ap_ar["ar"],
                    },
                    {
                        "dataset": "test",
                        f"AP@[IoBB = 0.50]": test_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": test_ap_ar["ar"],
                    },
                    {
                        "dataset": "val",
                        f"AP@[IoBB = 0.50]": val_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": val_ap_ar["ar"],
                    },
                    {
                        "dataset": "our",
                        f"AP@[IoBB = 0.50]": our_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": our_ap_ar["ar"],
                    },
                ]
            )

            df.to_csv(
                os.path.join(
                    "eval_results",
                    f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
                )
            )
            print_title(disease_str)
            print(df)


  0%|          | 0/4 [00:00<?, ?it/s]

Load custom model
Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
Mask Hidden Layers 256
Using SGD as optimizer with lr=0.001


  0%|          | 0/4 [01:07<?, ?it/s]


KeyboardInterrupt: 